# Arquitetura

![kafka-architecture](https://s3-sa-east-1.amazonaws.com/lcpi/032ec305-71c0-4b07-8f87-f56849c021c4.png)

## Servidor e broker

Para começarmos a explorar o Kafka, precisamos o inicializar os servidores antes!

### Start Zookeper
```bash
    $KAFKA_HOME/bin/zookeeper-server-start.sh $KAFKA_HOME/config/zookeeper.properties
```

### Start Kafka server
```bash
    $KAFKA_HOME/bin/kafka-server-start.sh $KAFKA_HOME/config/server.properties
```

## Tópico (`Topic`) 

Instanciado e gerenciado pela classe `KafkaAdminClient` através dos métodos:

- `list_topics`: Lista os tópicos no Kafka.

- `describe_topics`: Descreve os tópicos no Kafka.

- `create_topic`: Cria um novo tópico no Kafka.

- `delete_topic`: Deleta um tópico existente no Kafka.


---
`list_topics`
```python
from kafka.admin import KafkaAdminClient

admin_client = KafkaAdminClient(
    bootstrap_servers=['localhost:9092'],
)

topic_metadata = admin_client.list_topics()
print(topic_metadata)
```

---
`describe_topic`
```python
from kafka.admin import KafkaAdminClient

# Cria uma instância do KafkaAdminClient
admin_client = KafkaAdminClient(
    bootstrap_servers='localhost:9092',
)

# Define o nome do tópico a ser descrito
topic_name = admin_client.list_topics()

# Descreve o tópico
topic_description = admin_client.describe_topics(topic_name)

print(topic_description)

# Exibe as informações do tópico
for topic in topic_description: #[0].items():
    print(f'Tópico: {topic["topic"]}')
    print(f'Número de partições: {len(topic["partitions"])}')
    print(f'Replicas: {topic["partitions"][0]["replicas"]}')
    print(f'Líder: {topic["partitions"][0]["leader"]}')
```

---
`create_topic`
```python
from kafka.admin import KafkaAdminClient, NewTopic

admin_client = KafkaAdminClient(
    bootstrap_servers=["localhost:9092"],
)

topic = NewTopic(
    name="my_topic",
    num_partitions=1,
    replication_factor=1
)

admin_client.create_topics([topic])
```


---
`delete_topic`

Esse método não é síncrono, ou seja, não aguarda a conclusão da remoção do tópico, mas retorna um objeto DeleteTopicsResult que pode ser usado para verificar o status da operação. É importante notar que, após a deleção de um tópico, os dados armazenados nesse tópico não podem ser recuperados, portanto, é uma operação que deve ser realizada com cuidado.

```python
from kafka.admin import KafkaAdminClient, NewTopic

admin_client = KafkaAdminClient(
    bootstrap_servers=["localhost:9092"]
)

topics_to_delete = ['my_topic', 'meu_topico', 'my-topic', '__consumer_offsets']
delete_result =  admin_client.delete_topics(topics_to_delete)

print(delete_result)

```
---

### Configuração de tópicos

É possível configurar o tópico durante sua criação. Algumas opções são:

Além dos argumentos `name`, `num_partitions` e `replication_factor`, você pode passar os seguintes argumentos opcionais para a função `NewTopic`:

- `topic_configs`: um dicionário de configurações de tópico para serem definidas ao criar o tópico. As configurações possíveis incluem, por exemplo, "cleanup.policy", "retention.ms", "compression.type", etc.

- `replica_assignment`: uma lista de atribuições de réplicas personalizadas para cada partição, em vez de usar o comportamento padrão do Kafka de espalhar as réplicas uniformemente pelos brokers.

- `timeout_ms`: o tempo limite em milissegundos para criar o tópico. O padrão é 60000ms (1 minuto).

- `validate_only`: se definido como True, não cria o tópico, apenas valida a configuração do tópico. O padrão é False.

Você pode encontrar mais informações sobre a função `NewTopic` na [documentação do Kafka](https://kafka-python.readthedocs.io/en/latest/apidoc/KafkaAdminClient.html#kafka.admin.NewTopic).

O parâmetro `topic_configs` é um dicionário que pode ser usado para definir configurações específicas do tópico. Alguns dos parâmetros comuns que podem ser passados em `topic_configs` são:

- `cleanup_policy`: determina quando as mensagens antigas de um tópico devem ser removidas. Os valores possíveis são "delete" (para excluir mensagens antigas), "compact" (para compactar mensagens antigas) e "delete,compact" (para excluir e compactar mensagens antigas).

- `compression_type`: determina o tipo de compressão a ser usado para compactar as mensagens. Os valores possíveis são "none", "gzip", "snappy" e "lz4".

- `min_insync_replicas`: determina o número mínimo de réplicas que devem estar em sincronia antes que uma gravação possa ser considerada bem-sucedida. O valor padrão é 1, o que significa que uma gravação será considerada bem-sucedida assim que for gravada na réplica líder.

- `retention_bytes` e `retention_ms`: definem o tamanho máximo (em bytes) ou tempo máximo (em milissegundos) pelo qual as mensagens em um tópico serão mantidas antes de serem excluídas ou compactadas.

- `max_message_bytes`: define o tamanho máximo (em bytes) de uma única mensagem.

- `segment_bytes`: define o tamanho máximo (em bytes) de um segmento de log do tópico.

- `segment_ms`: define o tempo máximo (em milissegundos) pelo qual um segmento de log do tópico será mantido antes de ser fechado e arquivado.

- `message_timestamp_type`: define como o timestamp de uma mensagem é determinado. Os valores possíveis são "CreateTime" (o timestamp é determinado quando a mensagem é criada) e "LogAppendTime" (o timestamp é determinado quando a mensagem é gravada no log).

- `message_timestamp_difference_max_ms`: define o tempo máximo (em milissegundos) pelo qual o timestamp de uma mensagem pode ser maior do que o timestamp do produtor que a criou. Isso é usado para detectar relógios de sistema imprecisos em diferentes máquinas.

A lista completa de configurações disponíveis pode ser encontrada na [documentação do Kafka](https://kafka.apache.org/documentation/#topicconfigs).

Exemplo: 

---
`create_topic`
```python
topic = NewTopic(
    name="my_topic",
    num_partitions=3,
    replication_factor=2,
    topic_configs={
        "cleanup.policy": "compact",
        "retention.ms": "86400000"
    }
)
```



## Basics: producer & consumer

Para guiar o entendimento das funções e propriedades de cada componente do Kafka, vamos começar com um produtor e consumidor simples, gerar e receber algumas mensagens.

---
`producer`
---
```python
from kafka import KafkaProducer   # Importa a classe KafkaProducer da biblioteca kafka-python
import datetime as dt   # Importa a biblioteca datetime para trabalhar com datas e horas
import time   # Importa a biblioteca time para adicionar atrasos ao envio das mensagens

# Cria uma instância de um produtor Kafka e configura o endereço do servidor de bootstrap
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])   

# Define o nome do tópico para enviar as mensagens
topic_ = 'my_topic'   

# Cria uma mensagem
for i in range (20):
    # Captura a hora atual e converte em uma string formatada
    time_stamp = dt.datetime.strftime(dt.datetime.now(), format='%Y-%m-%d %H:%M:%S.%f')   
    # Monta uma mensagem com um carimbo de data/hora e um número de sequência
    message_ = f'{time_stamp} {i:6} Essa mensagem é enviada por KafkaProducer.'   


    # Imprime uma confirmação de mensagem enviada,
    print(f'Sent: {message_}')
    # Envia a mensagem ao tópico especificado pelo produtor
    producer.send(
        topic=topic_, 
        value=message_.encode('utf-8')
    )   
```

---
`consumer`
---
```python
# Importa a classe KafkaConsumer da biblioteca kafka-python
from kafka import KafkaConsumer

# Cria uma instância de um consumidor Kafka e configura o endereço do servidor de bootstrap e o nome do tópico a ser consumido
consumer = KafkaConsumer(
    'my_topic', 
    bootstrap_servers=['localhost:9092'],
)

# Itera sobre as mensagens recebidas pelo consumidor
for message in consumer:
    # Decodifica a mensagem e a imprime no console
    print(message.value.decode('utf-8'))
```
---

**Pergunta:** Todas as mensagens enviadas foram recebidas? O que pode estar errado nesse código simples?

# Produtor (`Producer`)


---

 Instanciado pelo `KafkaProducer` é uma classe da biblioteca `kafka-python` que permite aos usuários enviarem mensagens para tópicos em um cluster do Apache Kafka. As principais funções da classe `KafkaProducer` são:

- `send`: Envia uma mensagem para o Kafka.

- `flush`: Espera até que todas as mensagens enviadas sejam confirmadas pelo Kafka.

### flush & close

Garante que as mensagens sejam enviadas antes que o script se encerre e o produtor 'saia do ar'. Adicione essas duas linhas ao final do código e veja a diferença!

---
`produtor`

```python

# Força o envio de todas as mensagens pendentes antes de fechar o produtor.
producer.flush()   

# Fecha a conexão com o produtor
producer.close()
```
---


## Consumidor (`Consumer`) 

Instanciado pela classe `KafkaConsumer` da biblioteca `kafka-python`, permite a leitura de mensagens de um ou mais tópicos do Kafka. Suas principais funções são:

- `poll()` é um método utilizado para buscar novas mensagens em todos os tópicos e partições aos quais um consumidor está inscrito ou atribuído. Ele é usado em conjunto com um loop infinito para consumir continuamente mensagens. Ideal para batches!

- `subscribe()` é usado para inscrever um consumidor em um ou mais tópicos. O Kafka se encarrega de distribuir as partições desses tópicos entre os consumidores inscritos de acordo com o algoritmo de balanceamento de carga do grupo de consumidores. Ideal para streaming!


O método subscribe é mais adequado para quando se deseja receber todas as mensagens de um ou mais tópicos em tempo real, enquanto o poll é mais adequado para quando se deseja buscar ativamente as mensagens de um ou mais tópicos em intervalos regulares.

### poll

---
`consumer`

```python
from kafka import KafkaConsumer

consumer = KafkaConsumer(
    bootstrap_servers=['localhost:9092'],
)

consumer.subscribe(['my_topic'])

while True:
    
    # Lê as mensagens do Kafka em lotes, com um limite de X mensagens por lote.
    messages = consumer.poll(max_records=2, timeout_ms=3000)
    
    # Itera pelos lotes de mensagens lidos.
    for tp, msgs in messages.items():
        print(tp)
        
        # Itera pelas mensagens de cada lote.
        for msg in msgs:
            print(f"Received message: {msg.value.decode('utf-8')}")

    # Aguarda X segundos antes de ler o próximo lote de mensagens.
    time.sleep(3)
```
---

### subscribe

---
`consumer`
```python
from kafka import KafkaConsumer

consumer = KafkaConsumer(
    bootstrap_servers=['localhost:9092'],
)

consumer.subscribe(['my_topic'])

for message in consumer:
    print(f"Topic: {message.topic}")   # imprime o nome do tópico
    print(f"Partition: {message.partition}")   # imprime o número da partição em que a mensagem foi armazenada
    print(f"Offset: {message.offset}")   # imprime o deslocamento do registro na partição
    print(f"Timestamp: {message.timestamp}")   # imprime o carimbo de data/hora do registro
    print(f"Key: {message.key}")   # imprime a chave do registro, se houver
    print(f"Value: {message.value.decode('utf-8')}")   # imprime o valor da mensagem

```
---

## Partição (`Partition`)

É a unidade onde as mensagens são armazenadas e através delas são feitos os controles de leitura por parte dos consumidores. Esse controle é feito por *grupo de consumidores*.

#### OFFSET

É a posição de cada mensagem dentro de uma partição. Para consultar a posição do próximo registro a ser lido por um consumidor, usamos a função `position()`:

```python
current_offset = consumer.position(tp)
```

e para verificar o número de mensagens disponíveis usamos `end_offset()`:


```python
end_offset = consumer.end_offsets([tp])[tp]
```

---
`verify_offsets`
```python
from kafka import KafkaConsumer, TopicPartition

# Configurações do consumidor
consumer = KafkaConsumer(
    bootstrap_servers=['localhost:9092'],
    group_id=1,
    )


# # Define o tópico e a partição desejada
topic = 'teste_1'
partition = 0

# Definir a partição que deseja consultar
tp = TopicPartition(topic, partition)

# Atribuir a partição ao consumidor
consumer.assign([tp])

# Obter a posição atual do offset
current_offset = consumer.position(tp)

# Obter o offset mais recente da partição
end_offset = consumer.end_offsets([tp])[tp]

# Calcular a diferença entre a posição atual e o offset mais recente
unconsumed_messages = end_offset - current_offset

print(f"A posição atual do offset para a partição {partition} do tópico {topic} é {current_offset}")
print(f"{unconsumed_messages} mensagens não lidas na partição {tp}")
```
---

Essa operação também poder ser feita mp terminal, com o comando. Repare que esse comando deixa explicita a relação com o grupo de consumidores, que pode passar desapercebida no código python.

```bash
    $KAFKA_HOME/bin/kafka-consumer-groups.sh --bootstrap-server localhost:9092 --describe --group 1
```

## Consumer V2

Como discutimos na aula anterior, o servidor Kafka se limita a armazenar mensagens e metadados, enquanto o consumidor deve controlar o influxo de dados.

Vamos deixar nosso consumidor mais robusto, com as seguintes modificações:

- `enable_auto_commit`: esse parâmetro é responsável por habilitar ou desabilitar o recurso de commit automático do offset. Se for definido como `True`, o Kafka Consumer irá automaticamente enviar as confirmações de commit para o Kafka Broker de tempos em tempos. Caso contrário, o aplicativo terá que ser responsável por enviar os commits manualmente. O valor default é `True` e precisa ser alterado.

- `group_id`: esse parâmetro é usado para identificar qual grupo de consumidores o Kafka Consumer fará parte. Ele é obrigatório para a implementação de um consumidor de grupo. Todos os consumidores com o mesmo `group_id` pertencerão ao mesmo grupo de consumidores e receberão mensagens do mesmo conjunto de partições.
  
- `TopicPartition`: é uma classe utilizada para representar uma partição em um tópico específico.
  
- `seek()`: esse método é usado para configurar a posição de leitura em uma determinada partição. Ele permite que um Kafka Consumer comece a ler a partir de um offset específico em uma partição, em vez de ler todas as mensagens desde o último commit.

- `commit()`: esse método é usado para confirmar o commit do último offset lido pelo Kafka Consumer. Ele é usado quando `enable_auto_commit` é definido como `False` e o consumidor deve enviar manualmente as confirmações de commit.

---
`consumer`

```python
from kafka import KafkaConsumer, TopicPartition

# Configuração do consumidor
consumer = KafkaConsumer(
    bootstrap_servers=['localhost:9092'],
    enable_auto_commit=False,
    group_id = 1
)

# Atribuir a partição e o offset desejados
tp = TopicPartition('teste_2', 0)  # partição 0 de 'my_topic'
consumer.assign([tp])
consumer.seek(tp, 20)  # move para o offset X

# Loop para consumir mensagens a partir do offset definido
for message in consumer:
    print(f"Offset: {message.offset}, Chave: {message.key}, Valor: {message.value}")

    # commita o offset da última mensagem consumida para todas as partições atribuídas
    consumer.commit()

```
---

**Exercícios**

1. Envie mensagens para 5 partições diferentes de cada um de 3 tópicos.
2. Crie 6 consumidores distribua-os em 4 grupos.
3. Quando os produtores e consumidores estiverem funcionando, explore os metadados de consumo.
4. Faça testes e anote os pontos interessantes e suas dúvidas!

# Referências

## Producer

Instanciado pelo `KafkaProducer` é uma classe da biblioteca `kafka-python` que permite aos usuários produzir mensagens para tópicos em um cluster do Apache Kafka. As principais funções e métodos da classe `KafkaProducer` são:

1. `__init__(self, **configs)`: Inicializa uma instância da classe `KafkaProducer`. O método aceita argumentos-chave que podem ser usados para configurar o produtor. Alguns dos argumentos comuns são:

   - `bootstrap_servers`: Especifica a lista de endereços dos servidores Kafka. Por padrão, o valor é `localhost:9092`.

   - `acks`: Especifica o número de réplicas do cluster que confirmam o recebimento da mensagem antes que o produtor considere a mensagem enviada com sucesso. Os valores comuns são `0`, `1` ou `all`. O valor padrão é `1`.

   - `compression_type`: Especifica o tipo de compressão a ser usado para compactar as mensagens antes de enviá-las para o cluster. Os valores comuns são `none`, `gzip`, `snappy`, `lz4`, ou `zstd`. O valor padrão é `none`.

2. `send(self, topic, value=None, key=None, headers=None, partition=None, timestamp_ms=None)`: Envia uma mensagem para o tópico especificado. Os argumentos comuns são:

   - `topic`: O nome do tópico ao qual a mensagem será enviada.

   - `value`: O valor da mensagem. Deve ser serializado em bytes. Se `None`, uma mensagem vazia será enviada.

   - `key`: A chave da mensagem. Deve ser serializado em bytes. É usado para determinar a partição à qual a mensagem será enviada.

   - `headers`: Um dicionário de cabeçalhos personalizados para a mensagem.

   - `partition`: A partição para a qual a mensagem será enviada. Se `None`, o produtor selecionará automaticamente uma partição usando a chave da mensagem.

   - `timestamp_ms`: O tempo em milissegundos para associar à mensagem. Se `None`, o tempo atual será usado.

3. `flush(self, timeout=None)`: Espera todas as mensagens pendentes serem enviadas e confirmadas pelos servidores Kafka. O argumento `timeout` especifica o tempo máximo a aguardar em segundos. Se `None`, o método espera indefinidamente.

4. `close(self, timeout=None)`: Fecha o produtor e libera todos os recursos alocados. O argumento `timeout` especifica o tempo máximo a aguardar em segundos antes que o produtor seja encerrado. Se `None`, o método espera indefinidamente.

## Consumer

O `KafkaConsumer` é uma classe da biblioteca `kafka-python` que permite a leitura de mensagens de um ou mais tópicos do Kafka. Abaixo estão listadas suas principais funções e métodos:

- `__init__`: Método construtor que inicializa uma nova instância do consumidor Kafka. Os possíveis argumentos são:
  - `bootstrap_servers`: Endereço do(s) servidor(es) Kafka. Exemplo: `['localhost:9092', 'localhost:9093']`.
  - `group_id`: ID do grupo de consumidores ao qual este consumidor pertence. Os consumidores que têm o mesmo `group_id` compartilham a carga de trabalho da leitura das partições de um tópico. Se não especificado, será gerado um grupo com um nome aleatório.
  - `auto_offset_reset`: Define o que acontece quando não há offset inicial ou o offset não existe mais no servidor (exemplo: a partição foi excluída). Pode ser definido como `earliest` (começa a partir do início do tópico) ou `latest` (começa a partir da mensagem mais recente). O valor padrão é `latest`.
  - `enable_auto_commit`: Define se o Kafka commitará automaticamente o offset da última mensagem lida pelo consumidor. O valor padrão é `True`.
  - `key_deserializer`: Função ou classe usada para desserializar a chave da mensagem. O valor padrão é `None`.
  - `value_deserializer`: Função ou classe usada para desserializar o valor da mensagem. O valor padrão é `None`.
  - `fetch_max_wait_ms`: O tempo máximo em milissegundos que o consumidor aguardará por novas mensagens quando não houver nenhuma mensagem disponível naquele momento. O valor padrão é de 500 ms.
  - `max_poll_records`: O número máximo de registros que serão retornados a cada chamada `poll()`. O valor padrão é de 500 registros.



- `subscribe(topics, pattern=None, listener=None)` - subscreve um ou mais tópicos para consumo. É possível passar uma lista de strings com os nomes dos tópicos ou um objeto regex do módulo `re` para pattern. O argumento `listener` permite especificar um objeto `ConsumerRebalanceListener` personalizado para monitorar o balanceamento de partições entre os consumidores no grupo.
  
- `unsubscribe()` - cancela a assinatura de todos os tópicos atualmente assinados.
  
- `poll(timeout_ms=None, max_records=None, update_offsets=True)` - busca por registros disponíveis nos tópicos assinados e retorna um dicionário de partições e mensagens. O argumento `timeout_ms` define o tempo máximo que o método irá esperar por novas mensagens. O argumento `max_records` limita o número máximo de registros a serem retornados em uma chamada. O argumento `update_offsets` indica se o consumidor deve atualizar automaticamente os offsets das partições consumidas.
  
- `commit()` - confirma o processamento dos registros consumidos pelo consumidor. Este método confirma o último offset processado de cada partição. O consumidor só deve chamar esse método depois de processar todos os registros retornados por um poll. Se uma exceção ocorrer após o processamento de registros, o consumidor pode chamar o método `commit()` novamente com o mesmo `offset` para garantir que o registro seja confirmado.
  
- `commit_async()` - confirma o processamento dos registros consumidos pelo consumidor de forma assíncrona. Este método confirma o último offset processado de cada partição. O consumidor só deve chamar esse método depois de processar todos os registros retornados por um poll. Se uma exceção ocorrer após o processamento de registros, o consumidor pode chamar o método `commit_async()` novamente com o mesmo `offset` para garantir que o registro seja confirmado.
  
- `seek(partition, offset)` - altera a posição do consumidor em uma partição para um determinado offset. O argumento `partition` é um objeto `TopicPartition` que identifica a partição a ser alterada, e o argumento `offset` é o novo offset para o consumidor começar a ler a partir da partição especificada.
  
- `seek_to_beginning(partitions=None)` - altera a posição do consumidor para o início da partição especificada ou para todas as partições, se nenhum argumento for fornecido.
  
- `seek_to_end(partitions=None)` - altera a posição do consumidor para o final da partição especificada ou para todas as partições, se nenhum argumento for fornecido.
- `assignment()` - retorna uma lista de partições atualmente atribuídas a este consumidor.
  
- `pause(partitions=None)` - pausa o consumo de registros em uma ou mais partições. O argumento `partitions` é uma lista de objetos `TopicPartition` para as partições a serem pausadas. Se nenhum argumento for fornecido, todas as partições assinadas pelo consumidor serão pausadas.
  
- `resume(partitions=None)` - retoma o consumo de registros em uma ou mais partições que foram pausadas anteriormente. O argumento partitions é opcional e pode ser usado para especificar as partições específicas do tópico que devem ter o consumo retomado. Se partitions não for fornecido ou for definido como None, o consumo será retomado em todas as partições do tópico. Se uma lista de partições for fornecida, o consumo será retomado apenas nas partições listadas. O formato da lista de partições é uma lista de objetos TopicPartition.